In [101]:
import os
import sys
import numpy as np 
import open3d as o3d
import matplotlib.pyplot as plt
import pandas as pd
import tensorflow as tf
import collections

In [209]:
df = pd.read_pickle("../final_dataset.pkl") 
df.reset_index(inplace=True, drop=True)
df.head()

,id,voxelgrids,positions,poses,odoms
0,-1,"[[[25, 53], [32, 2], [17, 54], [26, 50], [30, ...","[[0.7827229554855963, -0.6603078958156333, 4.5...","[[[31.0, 57.0, 0.1766081601381302], [27.0, 52....","[[0.00022100093561436428, -0.00164606522993054..."
1,-1,"[[[17, 38], [36, 71], [18, 39], [29, 53], [24,...","[[1.8763295554092785, -0.7392371475816817, 8.6...","[[[17.0, 1.0, 0.22867129743099213], [17.0, 1.0...","[[-0.0010584275078791316, 0.005180639174413457..."
2,-1,"[[[13, 56], [8, 46], [20, 66], [44, 3], [43, 6...","[[1.2298735958886313, -0.6599135136166747, 8.2...","[[[19.0, 13.0, 0.1444173902273178], [17.0, 11....","[[-0.017197470816407222, -0.008501126993886861..."
3,-1,"[[[29, 68], [66, 59], [11, 93], [73, 58], [24,...","[[0.49709940835786537, -0.62631209064576, 7.09...","[[[18.0, 5.0, 0.17500989139080048], [21.0, 2.0...","[[0.00810160200217194, -0.006334385323650088, ..."
4,-1,"[[[22, 6], [23, 14], [23, 15], [26, 21], [23, ...","[[0.9673029278573837, -0.3436083278432631, 2.9...","[[[62.0, 62.0, 0.14962413907051086], [73.0, 45...","[[-0.027349297901359395, -0.011966854241542754..."


In [210]:
dataset_list = []
for index, row in df.iterrows():
    if row['positions'].shape[0] >= 20:
        pos_data = row['positions']
        poses_data = row['poses'].reshape(-1,17*3)
        vox_data = row['voxelgrids'].reshape(-1,1000*2)
        data_df = tf.data.Dataset.from_tensor_slices({"input_pos": pos_data, "input_poses":poses_data,"input_grid": vox_data})
        #print(data_df)
        data_df = data_df.window(20, shift=3, drop_remainder=True)
        print(data_df)
        for windows in data_df:
            print(windows)
            print(tf.data.Dataset.from_tensors(windows))
            dataset_list.append(tf.data.Dataset.from_tensors(windows))
            

<_WindowDataset element_spec={'input_pos': DatasetSpec(TensorSpec(shape=(3,), dtype=tf.float64, name=None), TensorShape([])), 'input_poses': DatasetSpec(TensorSpec(shape=(51,), dtype=tf.float64, name=None), TensorShape([])), 'input_grid': DatasetSpec(TensorSpec(shape=(2000,), dtype=tf.int32, name=None), TensorShape([]))}>
{'input_pos': <_VariantDataset element_spec=TensorSpec(shape=(3,), dtype=tf.float64, name=None)>, 'input_poses': <_VariantDataset element_spec=TensorSpec(shape=(51,), dtype=tf.float64, name=None)>, 'input_grid': <_VariantDataset element_spec=TensorSpec(shape=(2000,), dtype=tf.int32, name=None)>}
<_TensorDataset element_spec={'input_pos': DatasetSpec(TensorSpec(shape=(3,), dtype=tf.float64, name=None), TensorShape([])), 'input_poses': DatasetSpec(TensorSpec(shape=(51,), dtype=tf.float64, name=None), TensorShape([])), 'input_grid': DatasetSpec(TensorSpec(shape=(2000,), dtype=tf.int32, name=None), TensorShape([]))}>
<_WindowDataset element_spec={'input_pos': DatasetSpec(

In [211]:
dataset = tf.data.Dataset.from_tensor_slices(dataset_list)

In [212]:
for row in dataset.take(3):
  p = row.get_single_element()['input_pos']
  counter = 0
  for element in p.as_numpy_iterator():
    counter += 1
  print(counter)
  print(row)


20
<_VariantDataset element_spec={'input_pos': DatasetSpec(TensorSpec(shape=(3,), dtype=tf.float64, name=None), TensorShape([])), 'input_poses': DatasetSpec(TensorSpec(shape=(51,), dtype=tf.float64, name=None), TensorShape([])), 'input_grid': DatasetSpec(TensorSpec(shape=(2000,), dtype=tf.int32, name=None), TensorShape([]))}>
20
<_VariantDataset element_spec={'input_pos': DatasetSpec(TensorSpec(shape=(3,), dtype=tf.float64, name=None), TensorShape([])), 'input_poses': DatasetSpec(TensorSpec(shape=(51,), dtype=tf.float64, name=None), TensorShape([])), 'input_grid': DatasetSpec(TensorSpec(shape=(2000,), dtype=tf.int32, name=None), TensorShape([]))}>
20
<_VariantDataset element_spec={'input_pos': DatasetSpec(TensorSpec(shape=(3,), dtype=tf.float64, name=None), TensorShape([])), 'input_poses': DatasetSpec(TensorSpec(shape=(51,), dtype=tf.float64, name=None), TensorShape([])), 'input_grid': DatasetSpec(TensorSpec(shape=(2000,), dtype=tf.int32, name=None), TensorShape([]))}>


2024-06-13 14:47:54.744292: W tensorflow/core/framework/dataset.cc:956] Input of Window will not be optimized because the dataset does not implement the AsGraphDefInternal() method needed to apply optimizations.


In [213]:
len(dataset)

1857

In [214]:
# shuffle, batch and split dataset
SEED = 42
batch_size = 32

full_dataset = dataset.shuffle(100,SEED).batch(batch_size)

DATASET_SIZE = len(full_dataset)

train_size = int(0.7 * DATASET_SIZE)
val_size = int(0.15 * DATASET_SIZE)
test_size = int(0.15 * DATASET_SIZE)

train_dataset = full_dataset.take(train_size)
test_dataset = full_dataset.skip(train_size)
val_dataset = test_dataset.skip(val_size)
test_dataset = test_dataset.take(test_size)

In [218]:
len(train_dataset)
# 26 batches for len 20 and shift 5
# 41 batches for len 20 and shift 3


41

In [216]:
# create mask array, True= needs to be predicted
mask_arrays = []
for i in range(len(dataset)):
    mask_arr = [False] * 10 + [True] *10
    # hide 0-3 in between steps (for lazyness whole datapoint)
    hidden_nr = np.random.randint(4)
    hidden_idx = np.random.choice(range(10),hidden_nr, replace=False)
    for i in hidden_idx:
        mask_arr[i] = True
    mask_arrays.append(mask_arr)
len(mask_arrays)
# transform to match with batch and training, test and val datasets

1857